In [ ]:
results = []

for idx, row in treatment_events.iterrows():
    treated_nhood = row['neighborhood']
    treatment_date = int(row['yymm'])
    change_direction = np.sign(row['score_diff'])

    min_yymm = 1206
    max_yymm = 2503

    nhood_df = zhvi_transit[zhvi_transit['neighborhood'] == treated_nhood].copy()
    nhood_df = nhood_df.sort_values('yymm')

    previous_change = nhood_df[(nhood_df['yymm'].astype(int) < treatment_date) & (nhood_df['score_diff'] != 0)]
    if not previous_change.empty:
        min_pre_date = int(previous_change.iloc[-1]['yymm'])
    else:
        min_pre_date = shift_yymm(treatment_date, -18)

    max_post_date = shift_yymm(treatment_date, 6)
    max_post_date = min(max_post_date, max_yymm)
    min_pre_date = max(min_pre_date, min_yymm)

    analysis_period = zhvi_transit[(zhvi_transit['yymm'].astype(int) >= min_pre_date) & 
                                   (zhvi_transit['yymm'].astype(int) <= max_post_date)].copy()

    pre_score = row['trans_score'] - row['score_diff']

    def score_stable_in_window(g):
        scores = g[(g['yymm'].astype(int) >= min_pre_date) & (g['yymm'].astype(int) <= max_post_date)]['trans_score']
        return scores.nunique() == 1

    stable_nhoods = zhvi_transit.groupby('neighborhood').filter(score_stable_in_window)['neighborhood'].unique()

    control_scores = zhvi_transit[(zhvi_transit['neighborhood'].isin(stable_nhoods)) & 
                                  (zhvi_transit['yymm'].astype(int) == min_pre_date)].copy()

    try:
        pre_ZHVI = zhvi_transit[(zhvi_transit['neighborhood'] == treated_nhood) & 
                                (zhvi_transit['yymm'].astype(int) == min_pre_date)]['ZHVI'].values[0]
        treated_geom = zhvi_transit[(zhvi_transit['neighborhood'] == treated_nhood)].geometry.iloc[0]
    except IndexError:
        continue

    # Use centroid distances for valid spatial comparison
    treated_geom_centroid = treated_geom.centroid
    control_scores_geom_centroids = control_scores.geometry.centroid

    control_scores['ZHVI_diff'] = abs(control_scores['ZHVI'] - pre_ZHVI)
    control_scores['spatial_dist'] = control_scores_geom_centroids.distance(treated_geom_centroid)

    control_scores = control_scores[control_scores['ZHVI_diff'] < pre_ZHVI * 0.1]

    if change_direction > 0:
        candidate_controls = control_scores[control_scores['trans_score'] <= pre_score]
    else:
        candidate_controls = control_scores[control_scores['trans_score'] >= pre_score]

    candidate_controls = candidate_controls.sort_values(['ZHVI_diff', 'spatial_dist'])
    candidate_controls = candidate_controls.head(5)  # select top 5 closest in both ZHVI and spatial distance

    selected_nhoods = [treated_nhood] + candidate_controls['neighborhood'].tolist()
    sub_df = analysis_period[analysis_period['neighborhood'].isin(selected_nhoods)].copy()

    sub_df['treated'] = (sub_df['neighborhood'] == treated_nhood).astype(int)
    sub_df['post'] = (sub_df['yymm'].astype(int) >= treatment_date).astype(int)
    sub_df['treated_post'] = sub_df['treated'] * sub_df['post']

    def calc_slope(series):
        if len(series) < 7:
            return np.nan
        return series.diff(6)

    sub_df['trend'] = sub_df.groupby('neighborhood')['ZHVI'].transform(calc_slope)

    # Normalize sizerank so lower rank means more weight
    sub_df['weight'] = sub_df['SizeRank'].max() - sub_df['SizeRank'] + 1

    if sub_df['treated'].sum() < 3 or sub_df.shape[0] < 20 or candidate_controls.shape[0] == 0:
        continue

    try:
        model = smf.wls('ZHVI ~ treated + post + treated_post + trend + C(neighborhood) + C(yymm)', 
                        data=sub_df, weights=sub_df['weight']).fit()

        pval = model.pvalues.get('treated_post', np.nan)
        if pval < 0.05:
            results.append({
                'neighborhood': treated_nhood,
                'treatment_yymm': treatment_date,
                'coef': model.params.get('treated_post', np.nan),
                'pval': pval,
                'n_obs': sub_df.shape[0],
                'direction': 'increase' if change_direction > 0 else 'decrease'
            })
    except Exception as e:
        print(f"Error in regression for neighborhood {treated_nhood}: {e}")

# STEP 4: Combine results
results_df = pd.DataFrame(results)

# Print average treatment effects by direction
print("Average Treatment Effect (ATE) - Increases:", results_df[results_df['direction'] == 'increase']['coef'].mean())
print("Average Treatment Effect (ATE) - Decreases:", results_df[results_df['direction'] == 'decrease']['coef'].mean())
print(results_df[['neighborhood', 'treatment_yymm', 'coef', 'pval', 'n_obs', 'direction']])

Average Treatment Effect (ATE) - Increases: -39198.53072904378
Average Treatment Effect (ATE) - Decreases: -66583.31905317672
              neighborhood  treatment_yymm           coef          pval  \
0    AZALEA/HOLLYWOOD PARK            2206  -24606.816132  3.881020e-20   
1             BARRIO LOGAN            2106  -31091.044363  7.924671e-17   
2             BARRIO LOGAN            2206  -36825.126975  1.138003e-15   
3             BARRIO LOGAN            2306   -5213.831841  9.301305e-03   
4                 BAY PARK            2111  131040.211110  4.027200e-35   
..                     ...             ...            ...           ...   
232     UNIVERSITY HEIGHTS            1501   -9996.287903  1.505300e-02   
233     UNIVERSITY HEIGHTS            2206  -62584.924596  3.711024e-22   
234     UNIVERSITY HEIGHTS            2409  -42800.320342  1.132723e-15   
235          VALENCIA PARK            2206  -18888.676314  6.404274e-14   
236            WOODED AREA            1509   -774

In [ ]:
treatment_events = zhvi_transit[(zhvi_transit['score_diff'].notna()) & (zhvi_transit['score_diff'] != 0)]

# STEP 3: Loop through treatment events and estimate DiD for each
results = []

for idx, row in treatment_events.iterrows():
    treated_nhood = row['neighborhood']
    treatment_date = int(row['yymm'])
    change_direction = np.sign(row['score_diff'])

    min_yymm = 1206
    max_yymm = 2503

    nhood_df = zhvi_transit[zhvi_transit['neighborhood'] == treated_nhood].copy()
    nhood_df = nhood_df.sort_values('yymm')

    previous_change = nhood_df[(nhood_df['yymm'].astype(int) < treatment_date) & (nhood_df['score_diff'] != 0)]
    if not previous_change.empty:
        min_pre_date = int(previous_change.iloc[-1]['yymm'])
    else:
        min_pre_date = shift_yymm(treatment_date, -18)

    max_post_date = shift_yymm(treatment_date, 18)
    max_post_date = min(max_post_date, max_yymm)
    min_pre_date = max(min_pre_date, min_yymm)

    analysis_period = zhvi_transit[(zhvi_transit['yymm'].astype(int) >= min_pre_date) & 
                                   (zhvi_transit['yymm'].astype(int) <= max_post_date)].copy()

    pre_score = row['trans_score'] - row['score_diff']

    group_scores = analysis_period.groupby(['neighborhood'])['trans_score'].nunique().reset_index()
    static_nhoods = group_scores[group_scores['trans_score'] == 1]['neighborhood'].tolist()

    control_scores = zhvi_transit[(zhvi_transit['neighborhood'].isin(static_nhoods)) & 
                                  (zhvi_transit['yymm'].astype(int) == min_pre_date)].copy()

    try:
        pre_ZHVI = zhvi_transit[(zhvi_transit['neighborhood'] == treated_nhood) & 
                                (zhvi_transit['yymm'].astype(int) == min_pre_date)]['ZHVI'].values[0]
    except IndexError:
        continue

    control_scores['ZHVI_diff'] = abs(control_scores['ZHVI'] - pre_ZHVI)
    control_scores = control_scores[control_scores['ZHVI_diff'] < pre_ZHVI * 0.1]

    if change_direction > 0:
        candidate_controls = control_scores[control_scores['trans_score'] <= pre_score]['neighborhood'].tolist()
    else:
        candidate_controls = control_scores[control_scores['trans_score'] >= pre_score]['neighborhood'].tolist()

    selected_nhoods = [treated_nhood] + candidate_controls
    sub_df = analysis_period[analysis_period['neighborhood'].isin(selected_nhoods)].copy()

    sub_df['treated'] = (sub_df['neighborhood'] == treated_nhood).astype(int)
    sub_df['post'] = (sub_df['yymm'].astype(int) >= treatment_date).astype(int)
    sub_df['treated_post'] = sub_df['treated'] * sub_df['post']

    def calc_slope(series):
        if len(series) < 7:
            return np.nan
        return series.diff(6)

    sub_df['trend'] = sub_df.groupby('neighborhood')['ZHVI'].transform(calc_slope)

    if sub_df['treated'].sum() < 3 or sub_df.shape[0] < 20 or len(candidate_controls) == 0:
        continue

    try:
        model = smf.ols('ZHVI ~ treated + post + treated_post + trend + C(neighborhood) + C(yymm)', 
                        data=sub_df).fit()

        results.append({
            'neighborhood': treated_nhood,
            'treatment_yymm': treatment_date,
            'coef': model.params.get('treated_post', np.nan),
            'pval': model.pvalues.get('treated_post', np.nan),
            'n_obs': sub_df.shape[0],
            'direction': 'increase' if change_direction > 0 else 'decrease'
        })
    except Exception as e:
        print(f"Error in regression for neighborhood {treated_nhood}: {e}")

# STEP 4: Combine results
results_df = pd.DataFrame(results)

# Print average treatment effects by direction
print("Average Treatment Effect (ATE) - Increases:", results_df[results_df['direction'] == 'increase']['coef'].mean())
print("Average Treatment Effect (ATE) - Decreases:", results_df[results_df['direction'] == 'decrease']['coef'].mean())
print(results_df[['neighborhood', 'treatment_yymm', 'coef', 'pval', 'n_obs', 'direction']])

Average Treatment Effect (ATE) - Increases: -15142.47484637243
Average Treatment Effect (ATE) - Decreases: -32000.633047585427
       neighborhood  treatment_yymm          coef          pval  n_obs  \
0    ALLIED GARDENS            2206 -20821.157977  2.341047e-06    600   
1    ALLIED GARDENS            2409   2535.210435  3.169845e-01    132   
2        ALTA VISTA            2206  -4476.482238  2.128519e-02    400   
3        ALTA VISTA            2306   4704.371186  1.666711e-01    196   
4        ALTA VISTA            2501  90851.649914           NaN     42   
..              ...             ...           ...           ...    ...   
386   VALENCIA PARK            2306   -135.404920  9.659833e-01    168   
387   VALENCIA PARK            2409   2983.973849  2.932225e-02    132   
388     WOODED AREA            1509 -28092.560081  3.604549e-14    174   
389     WOODED AREA            2209  18530.390956  9.781887e-15     88   
390     WOODED AREA            2501  10897.376424          

In [ ]:
results = []
min_yymm = 1206
max_yymm = 2503
zhvi_transit = zhvi_transit.drop_duplicates(subset=['neighborhood', 'yymm'])

for idx, row in score_changes.iterrows():
    treated_nhood = row['neighborhood']
    treatment_date = int(row['yymm'])
    change_direction = np.sign(row['score_diff'])

    # Define date bounds
    min_yymm = 1206
    max_yymm = 2503

    # Get all dates for the treated neighborhood
    nhood_df = zhvi_transit[zhvi_transit['neighborhood'] == treated_nhood].copy()
    nhood_df = nhood_df.sort_values('yymm')

    # Find the previous date when score changed
    previous_change = nhood_df[(nhood_df['yymm'].astype(int) < treatment_date) & (nhood_df['score_diff'] != 0)]
    if not previous_change.empty:
        min_pre_date = int(previous_change.iloc[-1]['yymm'])
    else:
        min_pre_date = shift_yymm(treatment_date, -6)

    # Find the next date when score changes again
    next_change = nhood_df[(nhood_df['yymm'].astype(int) > treatment_date) & (nhood_df['score_diff'] != 0)]
    if not next_change.empty:
        max_post_date = int(next_change.iloc[0]['yymm'])
    else:
        max_post_date = shift_yymm(treatment_date, 6)

    # Ensure bounds are within overall date limits
    max_post_date = min(max_post_date, max_yymm)
    min_pre_date = max(min_pre_date, min_yymm)

    # Get analysis period between min_pre_date and max_post_date
    analysis_period = zhvi_transit[(zhvi_transit['yymm'].astype(int) >= min_pre_date) & 
                                   (zhvi_transit['yymm'].astype(int) <= max_post_date)].copy()

    # Get pre-treatment transit score for treated neighborhood
    pre_score = row['trans_score'] - row['score_diff']

    # Identify candidate control neighborhoods with no change in score during this period
    group_scores = analysis_period.groupby(['neighborhood'])['trans_score'].nunique().reset_index()
    static_nhoods = group_scores[group_scores['trans_score'] == 1]['neighborhood'].tolist()

    # Get score at min_pre_date for candidate control neighborhoods
    control_scores = zhvi_transit[(zhvi_transit['neighborhood'].isin(static_nhoods)) & 
                                  (zhvi_transit['yymm'].astype(int) == min_pre_date)].copy()

    # Filter based on direction of change
    if change_direction > 0:
        candidate_controls = control_scores[control_scores['trans_score'] <= pre_score]['neighborhood'].tolist()
    else:
        candidate_controls = control_scores[control_scores['trans_score'] >= pre_score]['neighborhood'].tolist()

    # Subset to treated + control neighborhoods
    selected_nhoods = [treated_nhood] + candidate_controls
    sub_df = analysis_period[analysis_period['neighborhood'].isin(selected_nhoods)].copy()

    # Mark treatment group
    sub_df['treated'] = (sub_df['neighborhood'] == treated_nhood).astype(int)

    # Mark post-treatment period
    sub_df['post'] = (sub_df['yymm'].astype(int) >= treatment_date).astype(int)

    # Create interaction term
    sub_df['treated_post'] = sub_df['treated'] * sub_df['post']

    sub_df['trend'] = sub_df.groupby('neighborhood')['ZHVI'].transform(lambda x: x.shift(1) - x.shift(2))

    # Skip if insufficient data
    if sub_df['treated'].sum() < 3 or sub_df.shape[0] < 20 or len(candidate_controls) == 0:
        continue

    try:
        # Run DiD regression with fixed effects, controlling for ZHVI level and trend
        model = smf.wls('ZHVI ~ treated + post + treated_post + trend + C(neighborhood) + C(yymm)', 
                        data=sub_df, 
                        weights=(sub_df['SizeRank'].max() - sub_df['SizeRank'] + 1)).fit()

        results.append({
            'neighborhood': treated_nhood,
            'treatment_yymm': treatment_date,
            'coef': model.params.get('treated_post', np.nan),
            'pval': model.pvalues.get('treated_post', np.nan),
            'n_obs': sub_df.shape[0],
            'direction': 'increase' if change_direction > 0 else 'decrease'
        })
    except Exception as e:
        print(f"Error in regression for neighborhood {treated_nhood}: {e}")

# STEP 4: Combine results
results_df = pd.DataFrame(results)

# Print average treatment effects by direction
print("Average Treatment Effect (ATE):", results_df['coef'].mean())
print("Average Treatment Effect (ATE) - Increases:", results_df[results_df['direction'] == 'increase']['coef'].mean())
print("Average Treatment Effect (ATE) - Decreases:", results_df[results_df['direction'] == 'decrease']['coef'].mean())
print(results_df[['neighborhood', 'treatment_yymm', 'coef', 'pval', 'n_obs', 'direction']])
    

In [ ]:
zhvi_transit

,neighborhood,yymm,ZHVI,trans_score,SizeRank,RegionID,no_change,no_transit,score_diff
0,ADAMS NORTH,1206,4.742081e+05,7.0,6927,403231,1,0,NaN
1,ADAMS NORTH,1207,4.806696e+05,7.0,6927,403231,1,0,0.0
2,ADAMS NORTH,1208,4.870897e+05,7.0,6927,403231,1,0,0.0
3,ADAMS NORTH,1209,4.931003e+05,7.0,6927,403231,1,0,0.0
4,ADAMS NORTH,1210,5.019519e+05,7.0,6927,403231,1,0,0.0
...,...,...,...,...,...,...,...,...,...
18375,WOODED AREA,2411,2.066945e+06,31.0,5371,276122,0,0,0.0
18376,WOODED AREA,2412,2.060732e+06,31.0,5371,276122,0,0,0.0
18377,WOODED AREA,2501,2.056097e+06,31.0,5371,276122,0,0,0.0
18378,WOODED AREA,2502,2.058843e+06,31.0,5371,276122,0,0,0.0
